In [1]:
import json
import requests
import pandas as pd
import mysql.connector

In [2]:
 mydb = mysql.connector.connect(
     host = "127.0.0.1",
     user = "root",
     passwd = "root",
     database = "crypto"
 )

In [3]:
mycursor = mydb.cursor(prepared=True)

In [4]:
symbol_list= ['ETH','EOS','XRP','BCH','LTC','TRX','ETC','BNB','OKB']

In [5]:
data_link = "https://min-api.cryptocompare.com/data/histominute?fsym=ETH&tsym=BTC&aggregate=15&limit=1"
response= requests.get(data_link)
x = response.json()
y = x["Data"]
time_ = y[1]['time']

In [6]:
time_

1570660200

In [7]:
# 15 min data
for k in range(20):
    for j in range(0,len(symbol_list)):
        data_link_1 = "https://min-api.cryptocompare.com/data/histominute?fsym="+symbol_list[j]+"&tsym=BTC&limit=100&toTs="+str(time_)+"&aggregate=15"
        response= requests.get(data_link_1)
        x = response.json()
        y = x["Data"]
        for i in range(0,len(y)):
            sql = """insert ignore into price (symbol,fromtimes,totimes,fromtime,totime,fromvolume,tovolume,open,high,low,close,processed) values(%s,%s,%s,from_unixtime(%s),from_unixtime(%s),%s,%s,%s,%s,%s,%s,%s)"""
            data = (symbol_list[j],y[i]['time']-900,y[i]['time'],y[i]['time']-900,y[i]['time'],y[i]['volumefrom'],y[i]['volumeto'],y[i]['open'],y[i]['high'],y[i]['low'],y[i]['close'],'0')
            mycursor.execute(sql,data)
            mydb.commit()
    try:
        time_=y[0]['time']
    except:
        print(time_)
        break

KeyboardInterrupt: 

In [8]:
# sorted by fromtime and symbol in table periodic_price;
for i in range(0,len(symbol_list)):
    sql = "SELECT * FROM price where symbol='"+symbol_list[i]+"' order by fromtime;"
    mycursor.execute(sql)
    records = mycursor.fetchall()
    for j in range(0,len(records)):
        mycursor.execute("""insert ignore into periodic_price (symbol,fromtimes,totimes,fromtime,totime,fromvolume,tovolume,open,high,low,close,processed) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",(symbol_list[i],records[j][-11],records[j][-10],records[j][-9],records[j][-8],records[j][-7],records[j][-6],records[j][-5],records[j][-4],records[j][-3],records[j][-2],'0'))
        mydb.commit()


KeyboardInterrupt: 

In [9]:
records

[(5455,
  'ETH',
  1569404700,
  1569405600,
  datetime.datetime(2019, 9, 25, 15, 15),
  datetime.datetime(2019, 9, 25, 15, 30),
  31050.580078125,
  621.3200073242188,
  0.0200399998575449,
  0.02005000039935112,
  0.019990000873804092,
  0.019990000873804092,
  0),
 (5456,
  'ETH',
  1569405600,
  1569406500,
  datetime.datetime(2019, 9, 25, 15, 30),
  datetime.datetime(2019, 9, 25, 15, 45),
  25245.619140625,
  504.57000732421875,
  0.019990000873804092,
  0.020010000094771385,
  0.01996999979019165,
  0.01996999979019165,
  0),
 (5457,
  'ETH',
  1569406500,
  1569407400,
  datetime.datetime(2019, 9, 25, 15, 45),
  datetime.datetime(2019, 9, 25, 16, 0),
  35648.3515625,
  711.3800048828125,
  0.01996999979019165,
  0.01998000033199787,
  0.019929999485611916,
  0.019950000569224358,
  0),
 (5458,
  'ETH',
  1569407400,
  1569408300,
  datetime.datetime(2019, 9, 25, 16, 0),
  datetime.datetime(2019, 9, 25, 16, 15),
  26039.419921875,
  520.5999755859375,
  0.019950000569224358,
  0.